# What we neglect
    boundary = 10
    i = 0
    while i < boundary:
        i += 1
        print i

# P4 Preview
* 如何將數值存到一個dictionary或list
* 範例：如何將AQX資料中的PM2.5欄位轉為整數後，產生SiteName至PM2.5的對應。
* 範例：如何每五分鐘儲存一次Youbike資料，且用時間作為Key，對應到所抓下來的資料？
* 如何將資料儲存為json檔
* 如何將資料儲存為csv檔

## Data Processing cycle
* 為了視覺化或使用別人開發的分析套件，通常必須要做點資料處理，有可能是把下載回來的資料轉換資料型態（例如文字轉為數字）好進行視覺化，或者因應像d3.js這類型的視覺化套件，而把下載下來的資料轉成符合d3.js輸入資料型態。步驟大約如下：
    1. 取得資料
    2. 清理和轉換資料
    3. 資料呈現和後續利用
        1. 把資料print出來，複製貼上到Excel或Google Spreadsheet中。
        2. 直接把資料用Python能用的視覺化套件例如[matplotlib](http://matplotlib.org)或[plot.ly](https://plot.ly)進行視覺化
        3. 將資料儲存為第三方視覺化軟體或套件所需要檔案型態和格式。


# Putting data into dictionary or list
## to dictionary: counting the frequency

In [11]:
alist = [1, 3, 2, 4, 1, 3, 1, 2, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 1, 4]
bdict = {}
for a in alist:
    if a not in bdict:
        bdict[a]  = 0
    bdict[a] += 1

for k, v in bdict.items():
    print "%d\t%d"%(k, v)

1	9
2	2
3	7
4	2


## General case: combine two equal length lists to a dictionary
    alist = [1, 2, 3, 4, 5, 6, 7]
    blist = [7, 6, 5, 4, 3, 2, 1]
    adict = {}
    for i in range(len(alist)):
        adict[alist[i]] = blist[i]
    print adict

## Example: AQ site to PM2.5 value: put data into dictionary
* 產生一個新的dictionary，將AQ SiteName對應到PM2.5的值。

In [5]:
import urllib
import json
url = 'http://opendata.epa.gov.tw/ws/Data/AQX/?format=json'
urllib.urlretrieve(url, "data.json")
data_str = open('data.json', 'r').read()
AQ_data = json.loads(data_str)
print AQ_data[0].keys()

[u'Status', u'PSI', u'CO', u'PM10', u'NO', u'MajorPollutant', u'WindDirec', u'FPMI', u'PublishTime', u'SO2', u'County', u'SiteName', u'WindSpeed', u'PM2.5', u'NOx', u'O3', u'NO2']


In [4]:
newdict = {}
for s in AQ_data:
    if s['PM2.5'] == "":
        PM25 = -1
    else:
        PM25 = int(s['PM2.5'])
    newdict[s['SiteName']] = PM25
print newdict

{u'\u53e4\u4ead': 30, u'\u9cf3\u5c71': 44, u'\u65b0\u5e97': 28, u'\u9f8d\u6f6d': 47, u'\u677e\u5c71': 24, u'\u524d\u93ae': 48, u'\u5b89\u5357': 62, u'\u967d\u660e': 24, u'\u6c50\u6b62': 28, u'\u5584\u5316': 49, u'\u81fa\u897f': 45, u'\u58eb\u6797': 30, u'\u6797\u5712': 51, u'\u6f6e\u5dde': 44, u'\u7f8e\u6fc3': 27, u'\u65b0\u838a': 29, u'\u6843\u5712': 30, u'\u81fa\u6771': 27, u'\u5b9c\u862d': 25, u'\u5609\u7fa9': 43, u'\u5927\u5712': 30, u'\u6c38\u548c': 25, u'\u4e8c\u6797': 54, u'\u6960\u6893': 53, u'\u5fe0\u660e': 46, u'\u57d4\u91cc': 33, u'\u95dc\u5c71': 25, u'\u89c0\u97f3': 43, u'\u6a4b\u982d': 40, u'\u99ac\u516c': 27, u'\u4e2d\u5c71': 33, u'\u5e73\u93ae': 35, u'\u982d\u4efd': 28, u'\u524d\u91d1': 48, u'\u5927\u540c': 29, u'\u65b0\u6e2f': 52, u'\u82b1\u84ee': 28, u'\u5c0f\u6e2f': 32, u'\u9ea5\u5bee': 51, u'\u5f70\u5316': 52, u'\u5d19\u80cc': 70, u'\u5fa9\u8208': 50, u'\u842c\u91cc': 26, u'\u4e2d\u58e2': 42, u'\u65b0\u71df': 47, u'\u6734\u5b50': 61, u'\u82d7\u6817': 46, u'\u571f\u57

## Example: AQ data into list by appending
* 現在的目的是要把AQ_data中的SiteName和PM2.5取出來，把PM2.5的值處理為整數後，和SiteName的資料產生一個temporary dictionary，再把這個temporary dictionary加到list的尾端。
* 所以思考邏輯是這樣：
    1. 先開一個空的list，稱為newdata。
    2. 對AQ_data中的每個站台都這麼做
        a. 取出站台的PM2.5，如果他為空的話，那新的PM2.5就是-1，不然就是PM2.5就是原先的字串轉整數。
        b. 建立一個temp_dict，其兩個keys為'SiteName'和'PM2.5'，分別對應到站台名稱和PM2.5的值。
        c. 把上述的temp_dict附加（append）到上述空的list之後

In [7]:
newdata = []
for s in AQ_data:
    if s['PM2.5'] == "":
        PM25 = -1
    else:
        PM25 = int(s['PM2.5'])
    temp_dict = {'SiteName': s['SiteName'],
                 'PM2.5': PM25}
    newdata.append(temp_dict)

## Example: Mapping ubike site name to its location

In [1]:
import urllib
import json
import gzip
url = 'http://data.taipei/youbike'
urllib.urlretrieve(url, "data.gz")
data_str = gzip.open('data.gz', 'r').read()
bike_data = json.loads(data_str)

* Method 1: Mapping each site name to its location represented by pair of values in list
        loc_dict = {'s1':[25.111, 114.111], 
                    's2':[25.112, 114.113]}

In [34]:
# print bike_data keys() here

# create a new dictionary loc_dict
for k, v in bike_data['retVal'].items():
    
# using for loop to v['sna'] to [lat, lng]

* Method 2: Mapping each site name to its location represented by two key-to-value pairs
        loc_dict = {'s1': {'lat':25.11111, 'lng':114.11111},
                    's2': {'lat':25.11111, 'lng':114.11111},
                   }

## Practice: Converting PM2.5 value to scale or degree and store them
* 上週的作業我請各位把PM2.5的值照網頁所述轉為1~10的視覺化程度（scale），並且照該表印出他是低度、中度、高度、和非常高度的污染程度（degree）。比較好的作法如下。我的想法是把scale的邊界值用list存下來，並且用for-loop去跑過一遍，看PM2.5的值落在哪一個區間，我就知道他的degree是多少。
* 現在我想產生一個dictionary，把每一個SiteName對應到一個dictionary，該dictionary存放著PM2.5的值、scale和degree。

In [47]:
# about the previous week
scaler = [-1, 11, 23, 35, 41, 47, 53, 58, 64, 70, 71]
cat = ["Unknown", "Low", "Low", "Low", "Mid", "Mid", "Mid", "High", "High", "High", "ExHigh"]
print len(scaler), len(cat)

AQ_dict = {}
for s in AQ_data:
    if s['PM2.5'] == "":
        PM25 = -1
    else:
        PM25 = int(s['PM2.5'])
    scale = 0
    for i in range(1, len(scaler)):
        if scaler[i-1] < PM25 <= scaler[i]:
            scale = i
    print "%s\t%d\t%d\t%s"%(s['SiteName'], PM25, scale, cat[scale])
    AQ_dict[s['SiteName']] = {'PM2.5': PM25, 'scale':scale, 'degree': cat[scale]}
# json.dump(AQ_dict, open('AQ.json', 'w'))

11 11
基隆	31	3	Low
汐止	27	3	Low
萬里	28	3	Low
新店	31	3	Low
土城	27	3	Low
板橋	31	3	Low
新莊	30	3	Low
菜寮	22	2	Low
林口	28	3	Low
淡水	31	3	Low
士林	34	3	Low
中山	30	3	Low
萬華	32	3	Low
古亭	26	3	Low
松山	32	3	Low
大同	31	3	Low
桃園	39	4	Mid
大園	24	3	Low
觀音	36	4	Mid
平鎮	-1	0	Unknown
龍潭	29	3	Low
湖口	32	3	Low
竹東	32	3	Low
新竹	27	3	Low
頭份	28	3	Low
苗栗	34	3	Low
三義	37	4	Mid
豐原	38	4	Mid
沙鹿	39	4	Mid
大里	29	3	Low
忠明	34	3	Low
西屯	35	3	Low
彰化	41	4	Mid
線西	41	4	Mid
二林	35	3	Low
南投	35	3	Low
斗六	44	5	Mid
崙背	43	5	Mid
新港	34	3	Low
朴子	39	4	Mid
臺西	30	3	Low
嘉義	39	4	Mid
新營	33	3	Low
善化	48	6	Mid
安南	41	4	Mid
臺南	34	3	Low
美濃	34	3	Low
橋頭	29	3	Low
仁武	34	3	Low
鳳山	28	3	Low
大寮	27	3	Low
林園	32	3	Low
楠梓	24	3	Low
左營	43	5	Mid
前金	35	3	Low
前鎮	34	3	Low
小港	33	3	Low
屏東	30	3	Low
潮州	24	3	Low
恆春	18	2	Low
臺東	21	2	Low
花蓮	25	3	Low
陽明	26	3	Low
宜蘭	25	3	Low
冬山	19	2	Low
三重	26	3	Low
中壢	36	4	Mid
竹山	38	4	Mid
永和	27	3	Low
復興	35	3	Low
埔里	23	2	Low
馬祖	35	3	Low
金門	33	3	Low
馬公	26	3	Low
關山	24	3	Low
麥寮	39	4	Mid


# Print out data for copy and paste
## Print out youbike site location and map to map
* 我希望能夠將每個Youbike站的站名和經緯度另存一個檔案或print出來，以利未來拿來繪製腳踏車站位置圖。由於腳踏車站沒有次序，所以我要產生一個dictionary（在底下稱為loc_dict），裡面每一個key-->value的對應，key就是腳踏車站的站名（sna），value是一個list，為腳踏車站的經緯度（e.g., [25.026808 121.546726]）。
    * 想法：撰寫一個for-loop將腳踏車站的經緯度取出來，再將腳踏車站的站名mapping到他的location。
* https://support.google.com/fusiontables/answer/2571232

In [9]:
for site, v in bike_data['retVal'].items():
    print v['sna'], v['lat'], v['lng']

# print out value separated by tab


捷運芝山站(2號出口) 25.10336 121.522629
捷運石牌站(2號出口) 25.114513 121.515677
國立臺北護理健康大學 25.118049 121.517512
國防大學 25.137976 121.493066
南港世貿公園 25.058 121.61422
臺灣師範大學(圖書館) 25.02665 121.52889
羅斯福新生南路口 25.01603085 121.5331757
蘭興公園 25.111839 121.525888
捷運行天宮站(3號出口) 25.059978 121.533302
捷運行天宮站(1號出口) 25.058369 121.532934
捷運東門站(4號出口) 25.0337 121.528988
臺大資訊大樓 25.02101 121.54153
辛亥新生路口 25.022413 121.53456
基隆長興路口 25.017054 121.544352
中崙高中 25.04878 121.56087
捷運六張犁站 25.023884 121.553161
中山天母路口 25.118843 121.529738
敦化基隆路口 25.022073 121.548336
羅斯福寧波東街口 25.031445 121.519411
洲子二號公園 25.079322 121.568688
東湖國中 25.073277 121.619521
文山行政中心 24.989902 121.569984
捷運大橋頭站(2號出口) 25.063404 121.512909
捷運動物園站(2號出口) 24.997659 121.578752
捷運木柵站 24.997747 121.574214
樹德公園 25.066688 121.516149
國立政治大學 24.988363 121.576536
士林運動中心 25.089175 121.521814
捷運士林站(2號出口) 25.092546 121.526556
北投運動中心 25.116665 121.509621
捷運明德站 25.110331 121.518316
內政部營建署 25.047805 121.545022
南港車站 25.05247 121.608202
龍門廣場 25.040901 121.548252
華西公園 25.038609 121.

# load and dump json file
* https://docs.python.org/2/library/json.html
* dump()和load()都是用來存檔或讀檔；dumps()和loads()是用來存取json型態的資料。若只是單純的檔案讀寫（I/O），那就用dump()和load()就好。
        json.dump(obj, fp, ...)
        json.dumps(obj, ...)
        json.load(fp, ...)
        json.loads(str, ...)

    

In [33]:
# print loc_dict
# 'w' stands for writing, while 'r' for reading
json.dump(loc_dict, open('ubike_loc_dict.json', 'w'))

# json.dump(loc_dict, 'ubike_loc_dict.json')

new_dict = json.load(open('ubike_loc_dict.json', 'r'))


In [36]:
with open('ubike_loc_dict.json', 'w') as outfile:
    json.dump(loc_dict, outfile)

with open('ubike_loc_dict.json', 'r') as infile:
    new_dict = json.load(infile)
    for k, v in new_dict.items():
        print k, v

北安大直街口 [25.084223, 121.549452]
松山家商 [25.036084, 121.579135]
捷運善導寺站(1號出口) [25.045267, 121.5222]
捷運士林站(2號出口) [25.092546, 121.526556]
福林公園 [25.096122, 121.530215]
林森公園 [25.052227, 121.525805]
市立美術館 [25.070629, 121.523268]
星雲金湖街口 [25.080915, 121.597339]
民生敦化路口 [25.057985, 121.548982]
捷運北投站 [25.132581, 121.498618]
捷運圓山站(2號出口) [25.071824, 121.519287]
捷運六張犁站 [25.023884, 121.553161]
捷運大湖公園站(2號出口) [25.083945, 121.601854]
木柵光輝路口 [24.988241, 121.55561]
臺北市立大學(天母校區) [25.113625, 121.53742]
台灣科技大學 [25.0131, 121.539723]
民生光復路口 [25.05862, 121.55514]
信義建國路口 [25.03293, 121.53747]
考試院 [24.987507, 121.549827]
國家圖書館 [25.037773, 121.517029]
龍江南京路口 [25.05298, 121.540568]
捷運科技大樓站 [25.025896, 121.543293]
捷運奇岩站 [25.126286, 121.500801]
植物園 [25.030015, 121.509813]
撫順公園 [25.064108, 121.522065]
南港世貿公園 [25.058, 121.61422]
臺北市客家文化主題公園 [25.02043, 121.525322]
建國農安街口 [25.065031, 121.536775]
捷運大直站(3號出口) [25.079278, 121.546683]
永樂市場 [25.054501, 121.510549]
捷運雙連站(2號出口) [25.057866, 121.520711]
建國和平路口 [25.02585, 121.537383]


## json.dump(data, io_file) to write data to file

In [32]:
outfile = open('AQdata.json', 'w')
json.dump(newdata, outfile)
outfile.close()

### Preferred style

In [33]:
with open('AQdata.json', 'w') as outfile:
    json.dump(newdata, outfile)

In [24]:
restore_data = json.load(open('AQdata.json', 'r'))

In [34]:
for s in restore_data[:10]:
    print s['SiteName'], s['PM2.5']

基隆 31
汐止 27
萬里 28
新店 31
土城 27
板橋 31
新莊 30
菜寮 22
林口 28
淡水 31


# Practice: Storing Youbike site data by mins

In [43]:
import urllib
import json
import gzip
import time
url = 'http://data.taipei/youbike'


bdata_list = []
for i in range(5):
    urllib.urlretrieve(url, "data.gz")
    bike_data = json.load(gzip.open('data.gz', 'r'))
    
    temp_dict = {}
    for k, v in bike_data['retVal'].items():
        temp_dict[v['sna']] = v
    
    bdata_list.append(temp_dict)
    
#     bdata_list.append(bike_data['retVal'])
    ## read time stamp
    
    time.sleep(3)
#     time.sleep(60)
with open('bike_data.json', 'w') as outfile:
    json.dump(bdata_list, outfile)
# print bike_data.keys()
# if bike_data['retCode'] == 1:
# #     print bike_data['retVal']
#     for k, v in bike_data['retVal'].items():
#         if float(v['sbi'])/int(v['tot']) > 0.7:
#             print k, v['sna'], v['sbi'], v['tot']
#             print "-"*30



In [44]:
with open('bike_data.json', 'r') as infile:
    merged_data = json.load(infile)
    print type(merged_data)
    for data in merged_data:
        pass

<type 'list'>


## Assignment AS04: Storing all youbike site location by dictionary

# Saving data to csv
* 除了儲存為json資料外，我有時候也希望將它儲存為xls檔，或者簡單的複製貼上到xls檔。

## print out data for copy-paste
* 你只要把它轉成欄位和欄位之間tab分隔（\t），那麼複製後就可以直接貼上到Excel或Google Spreadsheet。

In [46]:
import urllib
import json
url = 'http://opendata.epa.gov.tw/ws/Data/AQX/?format=json'
urllib.urlretrieve(url, "data.json")
data_str = open('data.json', 'r').read()
AQ_data = json.loads(data_str)

In [52]:
with open('PM25.csv', 'w') as outfile:
    for s in AQ_data:
        if s['PM2.5'] == '':
            PM25 = -1
        else:
            PM25 = int(s['PM2.5'])
        print "%s,%d"%(s['SiteName'], PM25)
        outfile.write("%s,%d\n"%(s['SiteName'].encode('big5'), PM25))

基隆,36
汐止,-1
萬里,36
新店,-1
土城,26
板橋,37
新莊,37
菜寮,30
林口,-1
淡水,32
士林,44
中山,45
萬華,38
古亭,-1
松山,40
大同,55
桃園,42
大園,43
觀音,-1
平鎮,-1
龍潭,44
湖口,39
竹東,31
新竹,34
頭份,31
苗栗,37
三義,-1
豐原,20
沙鹿,33
大里,14
忠明,22
西屯,26
彰化,35
線西,40
二林,35
南投,23
斗六,17
崙背,41
新港,39
朴子,38
臺西,41
嘉義,25
新營,43
善化,41
安南,45
臺南,48
美濃,27
橋頭,24
仁武,22
鳳山,27
大寮,23
林園,34
楠梓,37
左營,34
前金,28
前鎮,33
小港,25
屏東,12
潮州,41
恆春,21
臺東,24
花蓮,23
陽明,36
宜蘭,28
冬山,38
三重,44
中壢,42
竹山,20
永和,34
復興,37
埔里,18
馬祖,64
金門,55
馬公,27
關山,25
麥寮,39


In [50]:
# Add your column name here by print
for k, v in newdict2.items():
    print "%s\t%d\t%d\t%s"%(k, v['PM2.5'], v['scale'], v['degree'])

古亭	26	3	Low
鳳山	28	3	Low
新店	31	3	Low
龍潭	29	3	Low
松山	32	3	Low
前鎮	34	3	Low
安南	41	4	Mid
陽明	26	3	Low
汐止	27	3	Low
善化	48	6	Mid
臺西	30	3	Low
士林	34	3	Low
林園	32	3	Low
潮州	24	3	Low
美濃	34	3	Low
新莊	30	3	Low
桃園	39	4	Mid
臺東	21	2	Low
宜蘭	25	3	Low
嘉義	39	4	Mid
大園	24	3	Low
永和	27	3	Low
二林	35	3	Low
楠梓	24	3	Low
忠明	34	3	Low
埔里	23	2	Low
關山	24	3	Low
觀音	36	4	Mid
橋頭	29	3	Low
馬公	26	3	Low
中山	30	3	Low
平鎮	-1	0	Unknown
頭份	28	3	Low
前金	35	3	Low
大同	31	3	Low
新港	34	3	Low
花蓮	25	3	Low
小港	33	3	Low
麥寮	39	4	Mid
彰化	41	4	Mid
崙背	43	5	Mid
復興	35	3	Low
萬里	28	3	Low
中壢	36	4	Mid
新營	33	3	Low
朴子	39	4	Mid
苗栗	34	3	Low
土城	27	3	Low
南投	35	3	Low
金門	33	3	Low
淡水	31	3	Low
臺南	34	3	Low
竹東	32	3	Low
大寮	27	3	Low
屏東	30	3	Low
馬祖	35	3	Low
西屯	35	3	Low
湖口	32	3	Low
林口	28	3	Low
仁武	34	3	Low
萬華	32	3	Low
基隆	31	3	Low
新竹	27	3	Low
竹山	38	4	Mid
豐原	38	4	Mid
左營	43	5	Mid
三義	37	4	Mid
線西	41	4	Mid
板橋	31	3	Low
恆春	18	2	Low
沙鹿	39	4	Mid
冬山	19	2	Low
菜寮	22	2	Low
三重	26	3	Low
大里	29	3	Low
斗六	44	5	Mid


* 如果你要自己寫到檔案中，要自己在換行的地方寫入換行符號（\n）。
* 相同的作法也可寫入到tsv檔（tab分隔檔案）
* 寫得對的話，你的csv檔就可以直接用excel開啟。

In [59]:
with open('AQX.csv', 'w') as outfile:
    for k, v in newdict2.items():
#         print type(k)
#         outfile.write("%s\t%d\t%d\t%s\n"%(k.encode('big5'), v['PM2.5'], v['scale'], v['degree'])) # for excel
        outfile.write("%s\t%d\t%d\t%s\n"%(k.encode('utf8'), v['PM2.5'], v['scale'], v['degree'])) # for csv

# Others: function

In [37]:
def distance_on_unit_sphere(lat1, long1, lat2, long2):
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta, phi)
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc*6373
# print distance_on_unit_sphere(22.9979057, 120.22208048, 22.99774354, 120.22210156)